In [2]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, sample_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

C:\Users\Anneke\Anaconda3\lib\site-packages\keras\engine\training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.527464, acc.: 64.06%] [G loss: 0.681749]
1 [D loss: 0.422968, acc.: 65.62%] [G loss: 0.752087]
2 [D loss: 0.312141, acc.: 81.25%] [G loss: 0.960667]
3 [D loss: 0.262212, acc.: 90.62%] [G loss: 1.105689]
4 [D loss: 0.198627, acc.: 96.88%] [G loss: 1.258176]
5 [D loss: 0.181540, acc.: 98.44%] [G loss: 1.457715]
6 [D loss: 0.162550, acc.: 98.44%] [G loss: 1.589249]
7 [D loss: 0.130343, acc.: 98.44%] [G loss: 1.760429]
8 [D loss: 0.121955, acc.: 100.00%] [G loss: 1.974972]
9 [D loss: 0.093283, acc.: 100.00%] [G loss: 1.914315]
10 [D loss: 0.086662, acc.: 100.00%] [G loss: 1.991943]
11 [D loss: 0.080396, acc.: 100.00%] [G loss: 2.093877]
12 [D loss: 0.067028, acc.: 100.00%] [G loss: 2.130271]
13 [D loss: 0.081039, acc.: 100.00%] [G loss: 2.285628]
14 [D loss: 0.067291, acc.: 100.00%] [G loss: 2.321174]
15 [D loss: 0.065657, acc.: 100.00%] [G loss: 2.378483]
16 [D loss: 0.059575, acc.: 100.00%] [G loss: 2.550856]
17 [D loss: 0.054326, acc.: 100.00%] [G loss: 2.567084]
18 [D loss

151 [D loss: 0.029514, acc.: 100.00%] [G loss: 4.595066]
152 [D loss: 0.100955, acc.: 96.88%] [G loss: 5.077545]
153 [D loss: 0.047843, acc.: 100.00%] [G loss: 4.163254]
154 [D loss: 0.061714, acc.: 98.44%] [G loss: 3.152323]
155 [D loss: 0.112054, acc.: 95.31%] [G loss: 3.588294]
156 [D loss: 0.127837, acc.: 96.88%] [G loss: 3.536610]
157 [D loss: 0.158117, acc.: 92.19%] [G loss: 4.520093]
158 [D loss: 0.319414, acc.: 85.94%] [G loss: 4.431237]
159 [D loss: 0.142259, acc.: 93.75%] [G loss: 4.336802]
160 [D loss: 0.511322, acc.: 75.00%] [G loss: 4.103744]
161 [D loss: 0.091190, acc.: 100.00%] [G loss: 4.732767]
162 [D loss: 0.465593, acc.: 81.25%] [G loss: 3.273958]
163 [D loss: 0.057400, acc.: 100.00%] [G loss: 3.584258]
164 [D loss: 0.062756, acc.: 98.44%] [G loss: 3.872643]
165 [D loss: 0.111708, acc.: 95.31%] [G loss: 3.962339]
166 [D loss: 0.204302, acc.: 90.62%] [G loss: 3.665545]
167 [D loss: 0.147134, acc.: 95.31%] [G loss: 4.622077]
168 [D loss: 0.804108, acc.: 64.06%] [G loss

303 [D loss: 0.717924, acc.: 39.06%] [G loss: 0.752282]
304 [D loss: 0.751592, acc.: 40.62%] [G loss: 0.700918]
305 [D loss: 0.709804, acc.: 42.19%] [G loss: 0.725521]
306 [D loss: 0.688435, acc.: 45.31%] [G loss: 0.836386]
307 [D loss: 0.691541, acc.: 48.44%] [G loss: 0.789739]
308 [D loss: 0.700031, acc.: 46.88%] [G loss: 0.824576]
309 [D loss: 0.732678, acc.: 45.31%] [G loss: 0.735758]
310 [D loss: 0.720591, acc.: 43.75%] [G loss: 0.677880]
311 [D loss: 0.740735, acc.: 42.19%] [G loss: 0.702002]
312 [D loss: 0.722731, acc.: 45.31%] [G loss: 0.723341]
313 [D loss: 0.711187, acc.: 46.88%] [G loss: 0.696959]
314 [D loss: 0.674869, acc.: 53.12%] [G loss: 0.738283]
315 [D loss: 0.694049, acc.: 48.44%] [G loss: 0.773337]
316 [D loss: 0.731419, acc.: 42.19%] [G loss: 0.726184]
317 [D loss: 0.684670, acc.: 51.56%] [G loss: 0.741479]
318 [D loss: 0.704119, acc.: 43.75%] [G loss: 0.715622]
319 [D loss: 0.699050, acc.: 40.62%] [G loss: 0.707690]
320 [D loss: 0.700287, acc.: 48.44%] [G loss: 0.

...

29303 [D loss: 0.656124, acc.: 64.06%] [G loss: 1.054267]
29304 [D loss: 0.642422, acc.: 62.50%] [G loss: 1.056150]
29305 [D loss: 0.569907, acc.: 65.62%] [G loss: 0.957463]
29306 [D loss: 0.609879, acc.: 65.62%] [G loss: 1.089358]
29307 [D loss: 0.590804, acc.: 65.62%] [G loss: 1.118951]
29308 [D loss: 0.690592, acc.: 57.81%] [G loss: 0.952549]
29309 [D loss: 0.578896, acc.: 73.44%] [G loss: 1.070161]
29310 [D loss: 0.610804, acc.: 67.19%] [G loss: 0.996995]
29311 [D loss: 0.631500, acc.: 59.38%] [G loss: 1.024048]
29312 [D loss: 0.587193, acc.: 65.62%] [G loss: 0.994714]
29313 [D loss: 0.659666, acc.: 59.38%] [G loss: 1.015449]
29314 [D loss: 0.622106, acc.: 60.94%] [G loss: 1.089411]
29315 [D loss: 0.645850, acc.: 64.06%] [G loss: 1.136978]
29316 [D loss: 0.743329, acc.: 51.56%] [G loss: 1.002457]
29317 [D loss: 0.681969, acc.: 65.62%] [G loss: 0.952529]
29318 [D loss: 0.605710, acc.: 68.75%] [G loss: 1.031919]
29319 [D loss: 0.624780, acc.: 71.88%] [G loss: 0.986682]
29320 [D loss:

29449 [D loss: 0.642006, acc.: 65.62%] [G loss: 0.924773]
29450 [D loss: 0.648727, acc.: 64.06%] [G loss: 1.024404]
29451 [D loss: 0.616538, acc.: 60.94%] [G loss: 0.917369]
29452 [D loss: 0.633717, acc.: 57.81%] [G loss: 1.097569]
29453 [D loss: 0.617514, acc.: 67.19%] [G loss: 0.992076]
29454 [D loss: 0.655966, acc.: 62.50%] [G loss: 1.106263]
29455 [D loss: 0.538558, acc.: 68.75%] [G loss: 1.009665]
29456 [D loss: 0.618328, acc.: 64.06%] [G loss: 0.913380]
29457 [D loss: 0.564624, acc.: 70.31%] [G loss: 0.975756]
29458 [D loss: 0.555628, acc.: 82.81%] [G loss: 0.927500]
29459 [D loss: 0.578578, acc.: 71.88%] [G loss: 1.035806]
29460 [D loss: 0.590429, acc.: 71.88%] [G loss: 1.012794]
29461 [D loss: 0.672372, acc.: 51.56%] [G loss: 0.969735]
29462 [D loss: 0.518842, acc.: 76.56%] [G loss: 0.931919]
29463 [D loss: 0.672083, acc.: 60.94%] [G loss: 1.099245]
29464 [D loss: 0.557622, acc.: 73.44%] [G loss: 1.064448]
29465 [D loss: 0.600115, acc.: 73.44%] [G loss: 1.080167]
29466 [D loss:

29592 [D loss: 0.694407, acc.: 59.38%] [G loss: 0.927860]
29593 [D loss: 0.598245, acc.: 65.62%] [G loss: 0.926355]
29594 [D loss: 0.745862, acc.: 50.00%] [G loss: 0.892848]
29595 [D loss: 0.614673, acc.: 65.62%] [G loss: 0.920713]
29596 [D loss: 0.578361, acc.: 78.12%] [G loss: 1.083342]
29597 [D loss: 0.595417, acc.: 68.75%] [G loss: 1.030317]
29598 [D loss: 0.594307, acc.: 65.62%] [G loss: 1.098237]
29599 [D loss: 0.751220, acc.: 51.56%] [G loss: 1.005710]
29600 [D loss: 0.701435, acc.: 51.56%] [G loss: 1.014941]
29601 [D loss: 0.685169, acc.: 57.81%] [G loss: 0.862415]
29602 [D loss: 0.657511, acc.: 59.38%] [G loss: 1.065024]
29603 [D loss: 0.657643, acc.: 57.81%] [G loss: 0.912647]
29604 [D loss: 0.693367, acc.: 59.38%] [G loss: 0.910396]
29605 [D loss: 0.503495, acc.: 79.69%] [G loss: 1.008832]
29606 [D loss: 0.679677, acc.: 60.94%] [G loss: 1.024688]
29607 [D loss: 0.655229, acc.: 54.69%] [G loss: 0.937763]
29608 [D loss: 0.693543, acc.: 57.81%] [G loss: 0.944029]
29609 [D loss:

29739 [D loss: 0.595688, acc.: 67.19%] [G loss: 1.018264]
29740 [D loss: 0.626390, acc.: 62.50%] [G loss: 1.021280]
29741 [D loss: 0.675480, acc.: 56.25%] [G loss: 1.063685]
29742 [D loss: 0.722239, acc.: 54.69%] [G loss: 1.085340]
29743 [D loss: 0.554288, acc.: 78.12%] [G loss: 1.109269]
29744 [D loss: 0.660510, acc.: 59.38%] [G loss: 0.878049]
29745 [D loss: 0.758283, acc.: 56.25%] [G loss: 1.058866]
29746 [D loss: 0.638058, acc.: 64.06%] [G loss: 1.109154]
29747 [D loss: 0.743483, acc.: 51.56%] [G loss: 0.964343]
29748 [D loss: 0.565427, acc.: 71.88%] [G loss: 1.155653]
29749 [D loss: 0.623870, acc.: 64.06%] [G loss: 1.072188]
29750 [D loss: 0.626394, acc.: 60.94%] [G loss: 1.058797]
29751 [D loss: 0.674686, acc.: 53.12%] [G loss: 1.173178]
29752 [D loss: 0.579715, acc.: 68.75%] [G loss: 1.039092]
29753 [D loss: 0.559605, acc.: 67.19%] [G loss: 1.250556]
29754 [D loss: 0.694424, acc.: 59.38%] [G loss: 1.053129]
29755 [D loss: 0.661033, acc.: 56.25%] [G loss: 0.942078]
29756 [D loss:

29885 [D loss: 0.703865, acc.: 54.69%] [G loss: 1.105383]
29886 [D loss: 0.629838, acc.: 65.62%] [G loss: 1.027822]
29887 [D loss: 0.567098, acc.: 71.88%] [G loss: 1.026518]
29888 [D loss: 0.658276, acc.: 59.38%] [G loss: 0.955054]
29889 [D loss: 0.624914, acc.: 70.31%] [G loss: 1.052464]
29890 [D loss: 0.592179, acc.: 71.88%] [G loss: 1.015397]
29891 [D loss: 0.648680, acc.: 62.50%] [G loss: 1.042103]
29892 [D loss: 0.704602, acc.: 56.25%] [G loss: 0.978732]
29893 [D loss: 0.637410, acc.: 62.50%] [G loss: 1.155440]
29894 [D loss: 0.626284, acc.: 67.19%] [G loss: 1.034695]
29895 [D loss: 0.629185, acc.: 57.81%] [G loss: 1.061719]
29896 [D loss: 0.665832, acc.: 59.38%] [G loss: 0.913667]
29897 [D loss: 0.676451, acc.: 56.25%] [G loss: 0.941424]
29898 [D loss: 0.594487, acc.: 68.75%] [G loss: 1.054711]
29899 [D loss: 0.596525, acc.: 68.75%] [G loss: 1.027449]
29900 [D loss: 0.651303, acc.: 67.19%] [G loss: 1.042368]
29901 [D loss: 0.559376, acc.: 71.88%] [G loss: 1.054387]
29902 [D loss: